In [1]:
import pandas as pd
#pd.set_option("display.max_rows", None)
import numpy as np
import datetime
import time

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
pd.set_option('display.max_columns', None)

import plotly.graph_objects as go
import plotly.express as px
import plotly.figure_factory as ff
import dash
import dash_core_components as dcc
import dash_html_components as html

import ibmdata

In [2]:
def pull_wflevel(DAYSBACK=365):
    VMIN_CORNER = '_H2_0P50V_BLOCK'
    VNOM_CORNER = '_H2_0P75V_BLOCK'
    VMAX_CORNER = '_H2_1P05V_BLOCK'

    BLOCK_QUERY = f"""
    WITH cte AS(
        SELECT LEFT(lot_id, 5) AS lot_id_base, lot_id, wafer_id, family_code, MIN(tw.Last_test_date) AS date, tw.testProgramName AS step,
            --tw.testProgramName||RIGHT(parm_Label,8) AS label, 
            AVG(
                CASE WHEN RIGHT(tp.parm_Label,7) IN ('Block01','Block02','Block03','Block04', 'Block05') 
                THEN yield 
                ELSE null END
                ) AS Blk1to5,
            AVG(
                CASE WHEN RIGHT(tp.parm_Label,7) IN ('Block08','Block09','Block10','Block11','Block12','Block13','Block14','Block15','Block16','Block17','Block18') 
                THEN yield 
                ELSE null END
                ) AS Blk8to18
        FROM DMIW.PTileWaferFact ptwf
        INNER JOIN DMIW_SYSTEMS.TestParm tp ON ptwf.testparmkey = tp.testparmkey
        INNER JOIN DMIW_SYSTEMS.TestedWafer tw ON ptwf.testedWaferKey = tw.testedWaferKey
        WHERE tw.Last_test_date >= (current date - {DAYSBACK} days)
            AND tw.Tech_id = '7HPP'
            AND tw.Level = 'H2'
            AND tw.Calcdefs in ('SSL11', 'SSL21', 'SSL31')
            AND weighted_Mean IS NOT null AND ABS(weighted_Mean) < 1e25
            AND (UCASE(SUBSTR(tp.parm_Label,6,15))) LIKE '{VMIN_CORNER}'
            AND SUBSTR(tp.parm_Label,5,1) = RIGHT(tw.testProgramName,1)
        GROUP BY LEFT(lot_id, 5), lot_id, wafer_id, tw.testProgramName, family_code
        --,tw.testProgramName||RIGHT(parm_Label,8)
    )
    SELECT *, blk8to18 - blk1to5 AS delta
    FROM cte
    """
    Block_df = ibmdata.isdw.query(BLOCK_QUERY)
    Block_df['blk8to18'] = Block_df['blk8to18'].astype(float).round(2)
    Block_df['blk1to5'] = Block_df['blk1to5'].astype(float).round(2)
    Block_df['delta'] = Block_df['delta'].astype(float).round(2)
    return Block_df

In [3]:
df = pull_wflevel()

In [5]:
df.to_csv("~/Downloads/blks1to5.csv")

In [45]:
def pull_cplevel(DAYSBACK=365):
    VMIN_CORNER = '_H2_0P50V_BLOCK'
    VNOM_CORNER = '_H2_0P75V_BLOCK'
    VMAX_CORNER = '_H2_1P05V_BLOCK'
    QUERY = f"""
    WITH cte AS(
        SELECT LEFT(lot_id, 5) AS lot_id_base, lot_Id, wafer_Id, family_Code, tw.Calcdefs AS Sail, MIN(tw.Last_test_date) AS date,
            AVG(
                CASE WHEN RIGHT(tp.parm_Label,7) IN ('Block01','Block02','Block03','Block04', 'Block05') 
                THEN parmyield 
                ELSE null END
                ) AS Blk1to5,
            AVG(
                CASE WHEN RIGHT(tp.parm_Label,7) IN ('Block08','Block09','Block10','Block11','Block12','Block13','Block14','Block15','Block16','Block17','Block18') 
                THEN parmyield 
                ELSE null END
                ) AS Blk8to18
        FROM DMIW.ChipParmFactR cpfr
            JOIN DMIW_SYSTEMS.TestParm tp ON tp.testParmKey = cpfr.testParmKey
            JOIN DMIW_SYSTEMS.Geography g ON g.geographyKey = cpfr.geographyKey
            JOIN DMIW_SYSTEMS.TestedWafer tw ON tw.testedWaferKey = cpfr.testedWaferKey
        WHERE Last_test_date >= (current date - {DAYSBACK} days)
            AND Tech_id = '7HPP' AND Level = 'H2' AND tw.Calcdefs IN ('SSL21', 'SSL31')
            AND cpfr.parmValue IS NOT null AND ABS(parmValue) < 1e25 
            AND UCASE(SUBSTR(tp.parm_Label,6,15)) LIKE '{VMIN_CORNER}'
            AND SUBSTR(tp.parm_Label,5,1) = RIGHT(tw.testProgramName,1)
            AND radius_Center_5 NOT LIKE 'E'
        GROUP BY LEFT(lot_id, 5), lot_id, wafer_id, tw.Calcdefs, family_code
    )
    SELECT *, blk8to18 - blk1to5 AS delta
    FROM cte
    """
    df = ibmdata.isdw.query(QUERY)
    df['blk8to18'] = df['blk8to18'].astype(float).round(2)
    df['blk1to5'] = df['blk1to5'].astype(float).round(2)
    df['delta'] = df['delta'].astype(float).round(2)
    return df

In [47]:
df = pull_cplevel()

In [41]:
df1

,lot_id_base,lot_id,wafer_id,family_code,date,step,blk1to5,blk8to18,delta
0,A1AVP,A1AVPA.1,A1AVP-01,Q6,2021-05-08,SSL2,98.87,98.87,0.00
1,A1AVP,A1AVPA.1,A1AVP-01,Q6,2021-05-08,SSL3,97.80,98.40,0.60
2,A1AVP,A1AVPA.1,A1AVP-02,Q6,2021-05-08,SSL2,98.65,99.59,0.94
3,A1AVP,A1AVPA.1,A1AVP-02,Q6,2021-05-08,SSL3,100.00,99.90,-0.10
4,A1AVP,A1AVPA.1,A1AVP-03,Q6,2021-05-08,SSL2,100.00,99.90,-0.10
...,...,...,...,...,...,...,...,...,...
3209,AZAKD,AZAKD.1,AZAKD-15,Q6,2021-05-16,SSL2,99.77,99.90,0.12
3210,AZAKD,AZAKD.1,AZAKD-15,Q6,2021-05-16,SSL3,98.90,98.90,0.00
3211,AZB4Z,AZB4Z.1,AZB4Z-02,XQ,2021-05-08,SSL1,98.78,99.11,0.33
3212,AZB4Z,AZB4Z.1,AZB4Z-02,XQ,2021-05-08,SSL2,98.37,98.72,0.36


In [49]:
df.to_csv('~/Downloads/noedgeblk1to5.csv')

In [4]:
df

,lot_id_base,lot_id,wafer_id,family_code,date,step,blk1to5,blk8to18,delta
0,A1AVP,A1AVPA.1,A1AVP-01,Q6,2021-05-08,SSL2,98.87,98.87,0.00
1,A1AVP,A1AVPA.1,A1AVP-01,Q6,2021-05-08,SSL3,97.80,98.40,0.60
2,A1AVP,A1AVPA.1,A1AVP-02,Q6,2021-05-08,SSL2,98.65,99.59,0.94
3,A1AVP,A1AVPA.1,A1AVP-02,Q6,2021-05-08,SSL3,100.00,99.90,-0.10
4,A1AVP,A1AVPA.1,A1AVP-03,Q6,2021-05-08,SSL2,100.00,99.90,-0.10
...,...,...,...,...,...,...,...,...,...
4299,AZB51,AZB51.1,AZB51-09,XQ,2021-06-11,SSL2,99.16,98.95,-0.21
4300,AZB51,AZB51.1,AZB51-09,XQ,2021-06-11,SSL3,99.37,100.00,0.63
4301,AZB51,AZB51.1,AZB51-10,XQ,2021-06-11,SSL1,98.94,98.94,0.00
4302,AZB51,AZB51.1,AZB51-10,XQ,2021-06-11,SSL2,98.95,98.95,0.00
